<style>
r { color: Red }
o { color: Orange }
g { color: Green }
b { color: LightBlue}
w { color: White}
</style>

# **Mobile Robotics Presentation Group 16**
EPFL Course MICRO-452, fall 2023
- Gilles Regamey 296642
- Julien Droulet SCIPER
- Tom Rathjen    SCIPER
- Aubin Sabatier SCIPER

## Table of content
-   [Choices](#choices)
    - environement
    - filter
    - navigation
-   [Modules](#modules)
    - Vision
    - Global Navigation
    - Local Navigation
    - Filtering
    - Motion Control
-   [Demonstration](#demo)
    - setup
    - loop
-   [Results](#results)
    - encountered problems
    - performance
    - improvements



<a id="choices"></a>
# Choices taken
The project called for a environement map to navigate with fixed and un-planned obstacles to a destination point from any point. Fixed obstacles are constant trough the navigation and the robot can't use its sensor to detect them, they are only given by the camera at the start. Un-planned obstacles are detected only by the robot and can move at any time during navigation. 

## Environement
For the sake of simplicity, our map is a white A0 paper sheet with 4 <b>blue</b> box (4x4cm) at the corners. The Fixed obstacles are black pieces of cardboard of any shape. Mobile obstacles are <w>white</w> boxes of 4x4x4 cm 3d printed in PLA to have a more consistent reading with the proximity sensors. The robot is fitted with a paper hat with <r>red</r> circles in a isosceles triangle pattern for position and orientation. The destination is a big green circle 8cm in diameter. The image of the map is reframed to have a resolution of $1 \frac{px}{mm}$.

Below a theoretical map image before processing.

<img src="src\Vision\test_data\test_map.png" />

The fixed obstacles are abstracted in two ways for two different purposes:
- For Global navigation: occupancy grid of constant size.
- For Local navigation: polygonal contour approximation.
We'll see why in the modules descriptions.  

## Filtering
The camera gives us a pretty good estimation for the position and orientation of the robot, but we need to be able to navigate without it as somes frames can extract the position because of the changes in illumination or simply if the robot can't be seen. 

<r>**WHY A KALMAN ? - #[TODO]AUBIN**</r>

## Global navigation
For the path planning we choose the A\* Algorithm with the occupancy grid given by the vision module.
To have a margin to the obtacle 

<r>**why A\* - #[TODO]JULIEN**</r>

## Local navigation
Mobile and un-planned obstacles have to be detected and avoided. We choose to use a neural network system to be more robust to non standard obstacles. 

<r>**why neural #[TODO] TOM**</r>

<a id="modules"></a>
# Modules description

## **Vision**
The vision module is responsible for everything concerning the camera and vizualization.
Most importantly it has to detect the robot, the obstacles, the destination and the map correctly.
Visualization functions have been more of a tool to see and debug outputs of detection.
As our map elements are color based, we use the HLS (Hue Luminosity Saturation) encoding of the image to have a better chance at detecting color patches.
We use the opencv library extensively because of the blazingly fast parallel computation and all the usefull tools for detection and image processing.

### map detection
In the setup phase, we need to calibrate the camera to reframe the map in a consitant way to give a baseline for other functions. Opencv has a very usefull function called `warpPerspective()` just for that, we just need to give it the <w>warp matrix</w> and desired output shape. The function `vision.get_warp()` gives us this <w>warp matrix</w> from the image using the algorithm below and with the help of smaller functions called `reorder_border()`, `blob_point_list()` and another opencv function to compute the matrix from reference points `getPerspectiveTransform`. Note that this is not functionnal code, the real function and subsequent vision function can be found in `/src/vision.py`.

```python
def get_warp(capture_stream,ROI,padding,number_of_samples):
    while valid_samples < number_of_samples:
        frame = capture_stream.read()
        hsl_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL)
        nb_points,point_list = blob_point_list(Blue,CORNER_BLOB_FILT,CORNER_BLOB_VAL)
        if nb_points == 4:
            isvalid,ord_list = reorder_border(point_list)
            if isvalid:
                corners[:,:,smp] = ord_list
                valid_samples += 1
    avgCorn = mean(corners) 
     
    return cv2.getPerspectiveTransform(avgCorn,Sheetpts)
```
The `reorder_border()` function is just ordering the points relative to the center of the polygon they form (above/under,left/right) in order to give a consistant list of point for the averaging process.

Note that no action is taken for tangential and radial distortions. Opencv provides function for camera calibration and undistortion methods that need a calibration pattern (Charuco boards). We tried to use it but coulnd't get consitant camera matrix, it just made things worse most of the time so we decided to drop the idea knowing that robot position detection was good enough at our scale.

### Obstacle detection and abstraction

For the grid map of the obstacles we just apply a threshold to a scaled down filtered grayscale image of the map using opencv functions. You can also add a safe radius around obstacles using dilatation. 

```python
def get_grid_fixed_map(frame,shape,tresh=50,kernsz=5,robrad=80):
    kernel = np.ones((kernsz,kernsz),np.uint8)
    pxmap = cv2.inRange(frame,(0,0,0),(tresh,tresh,tresh))
    pxmap = cv2.morphologyEx(pxmap,cv2.MORPH_OPEN,kernel)
    if robrad != 0:
        safecircle = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(robrad,robrad))
        pxmap = cv2.dilate(pxmap,safecircle)
    temp = cv2.resize(pxmap, shape, interpolation=cv2.INTER_LINEAR)
    _,output = cv2.threshold(temp,10,1,type=cv2.THRESH_BINARY)
```

For the polygonal abstraction of the obstacles, we use opencv `findContours()` and `approxPolyDP()` to get an array of obstacles described as arrays of corner points.


```python
def get_obstacles(frame,tresh=50,eps=10,robrad=0):
    kernel = np.ones((5,5),np.uint8)
    pxmap = cv2.inRange(frame,(0,0,0),(tresh,tresh,tresh))
    pxmap = cv2.morphologyEx(pxmap,cv2.MORPH_OPEN,kernel)
    if robrad != 0:
        safecircle = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(robrad,robrad))
        pxmap = cv2.dilate(pxmap,safecircle)
    contp,hier =  cv2.findContours(pxmap,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    obstacles = []
    for cont in contp:
        cont = np.array(cont,dtype=np.float32).reshape((-1,2))
        scont = cv2.approxPolyDP(cont,eps,True)
        obstacles.append(scont)
    return obstacles
```


## **Module**

### submodule 1
description of module 

equation inline : $\alpha + \beta = e$

equations by line
\begin{equation}
S(\omega)=1.466\, H_s^2 \frac{\omega_0^5}{\omega^6} \exp\Bigl[-3^{\frac{\omega}{\omega_0}}\Bigr]^2
\end{equation}

### submodule 2
description of module 
code implementation
```python
if (goodboy):
    reward += 1
else:
    reward = 0
```
